In [1]:
import pandas as pd
from kaggle_pfs.data import readers

In [3]:
df = readers.items_by_month()
df.head()

,shop_id,item_id,0,1,2,3,4,5,6,7,...,24,25,26,27,28,29,30,31,32,33
0,48,21675,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,28,10501,1.0,0.0,2.0,3.0,2.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,18,8799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13,17091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,51,10542,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
test_set = pd.read_csv(readers.data_path("raw", "test.csv"))
test_set.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


A baseline model for time series is to just repeat the last value.

This is a way of saying *"...nothing changed, next month will be just like this one"*.
It's not very realistic but it sets an upper bound for the quality metric of the competition, $RMSE$.

In [5]:
repeat_last_month = (
    test_set.merge(df, on=["shop_id", "item_id"], how="left")
    .rename(columns={"33": "item_cnt_month"})
    .set_index("ID")
    .loc[:, "item_cnt_month"]
    .fillna(0)
    .clip(0, 20)
)
repeat_last_month.describe()

count    214200.000000
mean          0.255649
std           1.089856
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          20.000000
Name: item_cnt_month, dtype: float64

In [6]:
repeat_last_month.to_csv(readers.data_path("processed", "repeat_last_month.csv"))

Upon submission we get $1.16777$.